# Sprint 5 - DEMO
## Team Network Training

# 1 Datan esikäsittely ja manipulointi

## 1.1 Density-sensoreiden ongelmatapaus korjattu

Viime viikolla havaittu vinoumaa aiheuttava tiheyssensoreiden ongelmatapaus on nyt korjattu.

### 1.1.1 Viime viikon asetelma

![image](https://gitlab.dclabra.fi/wiki/uploads/upload_19908f73c3499bc2dcf4299792cac78c.png)

![image](https://gitlab.dclabra.fi/wiki/uploads/upload_2c10a50497f1310b63a07481176cd2f2.png)

### 1.1.2 Tilanne nyt - esimerkkiblokki

```python
# Block 1 
if (len(B1_density)<5):      
    if (len(B4_density) >= (len(B7_density))): 
        B1DensityAvg = B4_density['Density'].mean()
        return B1DensityAvg            
    else:
        B1DensityAvg = B7_density['Density'].mean()
        return B1DensityAvg
```

## 1.2 Jäljelle jäävät NaN-arvot ja väärään suuntaan "kutistuvat" arkit

Viime viikolla NaN-arvoja sisältävät rivit ja ylisuuren kutistuma-arvon sisältämät rivit pääsivät vielä läpi piirrematriisiin. Tämä on nyt korjattu piirrematriisin muodostusfunktiossa lisäämällä sinne tarpeelliset tarkistukset. Lopputulemana alkuperäisestä 146 rivistä päädyttiin 137 riviin.

### 1.2.1 Esimerkkikuva viime viikolla läpi päässeistä arvoista

![image](https://gitlab.dclabra.fi/wiki/uploads/upload_ad95e29d3c093877e3ed2214df719df4.png)

### 1.2.2 Tarkistukset piirrematriisia muodostettaessa

```python
# Checking for NaNs in current_row of feature matrix, if True don't append but skip (except for last row there is different behaviour)
if current_row.iloc[0].isna().any(axis=0) == True:
    nan_rows.append(peel_file)
    if printProgress == True: print(f"{i}/{amount_of_files}\tdropped nan-rows {len(nan_rows)}\tdropped too big shrinkage rows {len(shrinkage_above_one)}",end="\r")
    if i != length-1:
        continue
    elif i == length-1:
        skip_last_row = 1
        
######
######

# Sanity checking dryShrinkage, if doesn't pass don't append but skip (except for last row there is different behaviour)
if dry_data.iloc[0]['dryShrinkage'] > 1:
    shrinkage_above_one.append(dry_file)
    if printProgress == True: print(f"{i}/{amount_of_files}\tdropped nan-rows {len(nan_rows)}\tdropped too big shrinkage rows {len(shrinkage_above_one)}",end="\r")
    if i != length-1:
        continue
    elif i == length-1:
        skip_last_row = 1
```

## 1.3 Loppukosteuden ottaminen mukaan piirrematriisiin

Nyt kuiva-arkin kosteusprosentti on myös lisätty piirrematriisiin ja neuroverkon opetukseen mukaan.

![image](https://gitlab.dclabra.fi/wiki/uploads/upload_881892481ee4aa4fe15da82dad5e84dd.png)

## 1.4 traindevtest-sarake - varautuminen jättikokoisiin datasetteihin

Piirrematriisiin lisätään myös traindevtest-sarake, jolla varaudutaan todella suuriin datasetteihin. Eli jos datasetti ei mahdu muistiin kerralla, kuuluminen opetus-, testi- ja validointisetteihin määritetään tämän sarakkeen perusteella. Itse sarakkeen arvot eivät tule mukaan opetukseen.

```python
current_row['traindevtest']= np.random.choice([1,2,3],size=1,p=[0.8,0.1,0.1])
```

![image](https://gitlab.dclabra.fi/wiki/uploads/upload_1e4543993dcf893e027a5c82d7659c6e.png)

# 2 Tietokantaan puskeminen

![image](https://gitlab.dclabra.fi/wiki/uploads/upload_dfafad96beeb0efd68463cf15723805e.png)

- Integroitu osaksi full feature matrixia
- Puskee datan chunkeissa tai kaiken kerralla
    - Tällä hetkellä feature ja target -taulut erikseen, kun käytetään chunkitusta
    - Tulossa vaihtoehdoksi se, että kaikki menee samaan tauluun chunkituksessa
- Kakkostiimille on puskettu tietokantaan augmentoitu noin 500k setti
- Funktiot on mahdollista antaa kakkostiimille testattavaksi
- Tulossa (ehkä)
    - Augmentoinnin integrointi full feature matrixiin ja tietokantaan puskemiseen

Dokumentointi löytyy jälleen CodiMD:stä: [MariaDB-kontin pystyttäminen ja tietokantayhteyden muodostaminen](https://gitlab.dclabra.fi/wiki/Lhfjb-cJRFaSA_bvMb5eUg) ja [Kontin pystyttäminen Dockerfilen avulla](https://gitlab.dclabra.fi/wiki/QbEaq9AKSPmc3485Lk1mfw).

# 3 Neuroverkko

## 3.1 Funktio, jolla parametrien testailun voi tehdä systemaattisesti

- Neural_Network.py
    - Testerifunktio, jolla voi systemaattisesti testata eri parametrejä
    - Lähdettiin systemaattisesti testaamaan
        - Testattu suurelta osin augmentoidulla setillä
        - Original setillä alustavia tuloksia
    - Dropout lisättiin ihan vasta, ei vielä testejä sen kanssa
    - Kakkostiimille on helppo nakata tästä sitten tallennettu malli
- ipynb-tiedosto, jolla testit ajetaan
- Tallentaa tulokset csv-tiedostoihin
- CodiMD:ssä [taulukko](https://gitlab.dclabra.fi/wiki/U5V_dx7QRnq-Q4n29G11NQ), johon tuloksia kirjataan

## 3.2 Tuloksia taulukkoon

| Tester       | `epochs` | `loss/criterion` | `norm`       | `hidden_sizes`                        | `lr`       | `test_size` | `dropout` | `optimizer` | `val_mae`    | `val_mape`   | `val_mse`    |
| ------------ | -------- | ---------------- | ------------ | ------------------------------------- | ---------- | ----------- | --------- | ----------- | ------------ | ------------ | ------------ |
| **Original** | **5**    | **mse**          | **standard** | **[64,32,16]**                        | **0.001**  | **0.2**     |           | **rmsprop** | 0.004831     | 0.523477     | 0.000027     |
| Tuomas       | 5        | mse              | standard     | [64,32,16]                            | 0.001      | 0.2         |           | **Adam**    | 0.001033     | 0.112150     | 0.000002     |
| Tuomas       | 5        | mse              | standard     | [64,32,16]                            | 0.001      | 0.2         |           | SGD         | 0.024483     | 2.653333     | 0.001004     |
| Tuomas       | 5        | mse              | standard     | [64,32,16]                            | 0.001      | 0.2         |           | Adadelta    | 0.075339     | 8.173112     | 0.009347     |
| Tuomas       | 5        | mse              | standard     | [64,32,16]                            | 0.001      | 0.2         |           | Adagrad     | 0.023572     | 2.554444     | 0.000987     |
|              |          |                  |              |                                       |            |             |           |             |              |              |              |
| Anssi        | 5        | mse              | standard     | [64,32,16]                            | 0.001      | 0.2         |           | rmsprop     | 0.004831     | 0.523477     | 0.000027     |
| Anssi        | 5        | mse              | standard     | [128,64,32,16]                        | 0.001      | 0.2         |           | rmsprop     | 0.003005     | 0.326464     | 0.000011     |
| Anssi        | 5        | mse              | standard     | **[256,128,64,32,16]**                | 0.001      | 0.2         |           | rmsprop     | 0.001535     | 0.167503     | 0.000004     |
| Anssi        | 5        | mse              | standard     | [512,256,128,64,32,16]                | 0.001      | 0.2         |           | rmsprop     | 0.002315     | 0.250118     | 0.000007     |
| Anssi        | 5        | mse              | standard     | [1024,512,256,128,64,32,16]           | 0.001      | 0.2         |           | rmsprop     | 0.001811     | 0.196899     | 0.000005     |
| Anssi        | 5        | mse              | standard     | [2048,1024,512,256,128,64,32,16]      | 0.001      | 0.2         |           | rmsprop     | 0.002870     | 0.311318     | 0.000010     |
| Anssi        | 5        | mse              | standard     | [4096,2048,1024,512,256,128,64,32,16] | 0.001      | 0.2         |           | rmsprop     | 0.001951     | 0.211939     | 0.000005     |
| Anssi        | 5        | mse              | standard     | [256,128,64,32]                       | 0.001      | 0.2         |           | rmsprop     | 0.005735     | 0.624093     | 0.000050     |
| Anssi        | 5        | mse              | standard     | [128,128,64,32]                       | 0.001      | 0.2         |           | rmsprop     | 0.002507     | 0.275060     | 0.000024     |
| Anssi        | 5        | mse              | standard     | [128,128,64,32,16]                    | 0.001      | 0.2         |           | rmsprop     | 0.002823     | 0.306779     | 0.000010     |
| Anssi        | 5        | mse              | standard     | *[128,64,32,16,8]*                    | 0.001      | 0.2         |           | rmsprop     | 0.001798     | 0.195366     | 0.000005     |
| Anssi        | 5        | mse              | standard     | [32,16,8]                             | 0.001      | 0.2         |           | rmsprop     | 0.003178     | 0.345829     | 0.000014     |
| Anssi        | 5        | mse              | standard     | [64,32,16,8]                          | 0.001      | 0.2         |           | rmsprop     | 0.003460     | 0.374723     | 0.000014     |
| Anssi        | 5        | mse              | standard     | [128,64,32,16,8]                      | 0.001      | 0.2         |           | rmsprop     | 0.002270     | 0.246705     | 0.000007     |
| Anssi        | 5        | mse              | standard     | [256,256,128,64,32]                   | 0.001      | 0.2         |           | rmsprop     | 0.002257     | 0.245815     | 0.000007     |
| Anssi        | 5        | mse              | standard     | [512,256,128,64,32]                   | 0.001      | 0.2         |           | rmsprop     | 0.004002     | 0.433518     | 0.000021     |
|              |          |                  |              |                                       |            |             |           |             |              |              |              |
| Krista       | 5        | mse              | standard     | [64,32,16]                            | 0.008      | 0.2         |           | rmsprop     | 0.011143     | 1.231145     | 0.000352     |
| Krista       | 5        | mse              | standard     | [64,32,16]                            | 0.005      | 0.2         |           | rmsprop     | 0.004899     | 0.533809     | 0.000067     |
| Krista       | 5        | mse              | standard     | [64,32,16]                            | 0.002      | 0.2         |           | rmsprop     | 0.002690     | 0.296458     | 0.000053     |
| Krista       | 5        | mse              | standard     | [64,32,16]                            | 0.0018     | 0.2         |           | rmsprop     | 0.001436     | 0.157151     | 0.000005     |
| Krista       | 5        | mse              | standard     | [64,32,16]                            | 0.00175    | 0.2         |           | rmsprop     | 0.002422     | 0.262943     | 0.000009     |
| Krista       | 5        | mse              | standard     | [64,32,16]                            | **0.0017** | 0.2         |           | rmsprop     | 0.001270     | 0.138381     | 0.000004     |
| Krista       | 5        | mse              | standard     | [64,32,16]                            | 0.00165    | 0.2         |           | rmsprop     | 0.003694     | 0.401318     | 0.000017     |
| Krista       | 5        | mse              | standard     | [64,32,16]                            | 0.0016     | 0.2         |           | rmsprop     | 0.002894     | 0.314134     | 0.000012     |
| Krista       | 5        | mse              | standard     | [64,32,16]                            | 0.0015     | 0.2         |           | rmsprop     | 0.002234     | 0.242284     | 0.000007     |
| Krista       | 5        | mse              | standard     | [64,32,16]                            | 0.0009     | 0.2         |           | rmsprop     | 0.003447     | 0.372776     | 0.000014     |
| Krista       | 5        | mse              | standard     | [256,128,64,32,16]                    | 0.0015     | 0.2         |           | Adam        | 0.001070     | 0.116340     | 0.000002     |
|              |          |                  |              |                                       |            |             |           |             |              |              |              |
| Kosti        | 5        | mse              | minmax       | [64,32,16]                            | 0.001      | 0.2         |           | rmsprop     | 0.002200     | 0.235100     | 0.000007     |
| Kosti        | 5        | mae              | minmax       | [64,32,16]                            | 0.001      | 0.2         |           | rmsprop     | 0.001724     | 0.187597     | 0.000005     |
| Kosti        | 5        | mape             | minmax       | [64,32,16]                            | 0.001      | 0.2         |           | rmsprop     | 0.003130     | 0.340189     | 0.000013     |
| Kosti        | 5        | mae              | standard     | [64,32,16]                            | 0.001      | 0.2         |           | rmsprop     | 0.002991     | 0.324600     | 0.000012     |
| Kosti        | 5        | mape             | standard     | [64,32,16]                            | 0.001      | 0.2         |           | rmsprop     | 0.004034     | 0.436996     | 0.000023     |
| Kosti        | 5        | mae              | minmax       | [256, 128, 64,32,16]                  | 0.001      | 0.2         |           | rmsprop     | 0.002801     | 0.303454     | 0.000010     |
| Kosti        | 5        | mape             | minmax       | [256, 128, 64,32,16]                  | 0.001      | 0.2         |           | rmsprop     | 0.002590     | 0.280015     | 0.000009     |
| Kosti        | 5        | mape             | minmax       | [256, 128, 64,32,16]                  | 0.001      | 0.5         |           | Adam        | 0.001062     | 0.115386     | 0.000002     |
|              |          |                  |              |                                       |            |             |           |             |              |              |              |
| **Tuomas**   | **5**    | **mse**          | **standard** | **[256,128,64,32,16]**                | **0.001**  | **0.2**     |           | **Adam**    | **0.000945** | **0.102499** | **0.000001** |
| Anssi        | 5        | mse              | standard     | [128,64,32,16,8]                      | 0.001      | 0.2         |           | Adam        | 0.001061     | 0.114933     | 0.000002     |
| Tuomas       | 5        | mse              | standard     | [256,128,64,32,16]                    | 0.0017     | 0.2         |           | Adam        | 0.000976     | 0.105860     | 0.000001     |
| Anssi        | 5        | mse              | standard     | [128,64,32,16,8]                      | 0.0017     | 0.2         |           | Adam        | 0.001299     | 0.140530     | 0.000003     |
| Anssi        | 5        | mse              | minmax       | [256,128,64,32,16]                    | 0.001      | 0.2         |           | rmsprop     | 0.003843     | 0.416618     | 0.000016     |
| Anssi        | 5        | mse              | minmax       | [256,128,64,32,16]                    | 0.001      | 0.2         |           | Adam        | 0.001016     | 0.110251     | 0.000002     |
| Anssi        | 5        | mse              | standard     | [256,128,64,32,16]                    | 0.001      | 0.9         |           | Adam        | 0.002015	    | 0.218510	   | 0.000007     |
|              |          |                  |              |                                       |            |             |           |             |              |              |              |
| Kosti        | 5        | mse              | standard     | [256,128,64,32,16]                    | 0.0008     | 0.2         |           | Adam        | 0.001024     | 0.11036      | 0.000002     |
| Kosti        | 5        | mse              | standard     | [256,128,64,32,16]                    | 0.0009     | 0.2         |           | Adam        | 0.001016     | 0.109957     | 0.000001     |
| Kosti        | 5        | mse              | standard     | [256,128,64,32,16]                    | 0.0011     | 0.2         |           | Adam        | 0.000973     | 0.105818     | 0.000001     |
| Kosti        | 5        | mse              | standard     | [256,128,64,32,16]                    | 0.0012     | 0.2         |           | Adam        | 0.001013     | 0.109691     | 0.000001     |
| Kosti        | 5        | mse              | standard     | [256,128,64,32,16]                    | 0.0013     | 0.2         |           | Adam        | 0.001983     | 0.215015     | 0.000005	  |
| Kosti        | 5        | mse              | minmax       | [256,128,64,32,16]                    | 0.0008     | 0.2         |           | Adam        | 0.001283     | 0.139201     | 0.000003     |
| Kosti        | 5        | mse              | minmax       | [256,128,64,32,16]                    | 0.0009     | 0.2         |           | Adam        | 0.001019     | 0.110371     | 0.000002     |
| Kosti        | 5        | mse              | minmax       | [256,128,64,32,16]                    | 0.0011     | 0.2         |           | Adam        | 0.001043     | 0.113343     | 0.000002	  |
| Kosti        | 5        | mse              | minmax       | [256,128,64,32,16]                    | 0.0012     | 0.2         |           | Adam        | 0.001044	    | 0.113346     | 0.000002     |
| Kosti        | 5        | mse              | minmax       | [256,128,64,32,16]                    | 0.0013     | 0.2         |           | Adam        | 0.000955     | 0.103464     | 0.000001     |
|              |          |                  |              |                                       |            |             |           |             |              |              |              |
| Krista       | 5        | mse              | standard     | [256,128,64,32,16]                    | 0.001      | 0.2         | 0.2       | Adam        | 0.002442     | 0.270912     | 0.000024     |
| Krista       | 5        | mse              | standard     | [256,128,64,32,16]                    | 0.001      | 0.2         | 0.3       | Adam        | 0.006778     | 0.734300     | 0.000083     |
| Krista       | 5        | mse              | standard     | [256,128,64,32,16]                    | 0.001      | 0.2         | 0.4       | Adam        | 0.007380     | 0.819544     | 0.000219     |
| Krista       | 5        | mse              | standard     | [256,128,64,32,16]                    | 0.001      | 0.2         | 0.5       | Adam        | 0.009656     | 1.051882     | 0.000165     |
|              |          |                  |              |                                       |            |             |           |             |              |              |              |
|              |          |                  |              |                                       |            |             |           |             |              |              |              |
|              |          |                  |              |                                       |            |             |           |             |              |              |              |

## 3.3 Opetuksen jälkeen testaus alkuperäisellä datalla

### 3.3.1 Alkuperäinen data on ollut augmentoidun datan seassa mukana

![image](https://cdn.discordapp.com/attachments/898181430776037386/916277181309673502/unknown.png)

### 3.3.2 Alkuperäinen data ei ole ollut augmentoidun datan seassa mukana

![image](https://cdn.discordapp.com/attachments/898181430776037386/916277118248292373/unknown.png)

![image](https://cdn.discordapp.com/attachments/773834983960870952/916279333541269504/unknown.png)